### Imports

In [1]:
import numpy as np

### Generate random matrices 5x5

In [2]:
# Size of the matrices
N = 5

# Computation through batches allows us to save memory by asking the diagonal of the matrices to be bounded
batch = 10000
N_batches = 5
N_samples = N_batches * batch

# The parameter 'c' defines the bound on the eigevalues
c = 1

# The parameter 'random_bounds' defines the bound on parameters of the matrices
random_bounds = 1

In [3]:
def gen(N, N_samples):
    
    np.random.seed(417)
    
    m = np.random.uniform(-random_bounds, random_bounds,size=(batch, N, N))
    lst = (m + np.transpose(m, axes=(0,2,1)))/2
    
    lst = lst[np.all(np.abs(np.diagonal(lst, axis1 = 1, axis2 = 2)) < c, axis=1)]
    
    
    for _ in range(N_batches - 1):
        
        m = np.random.uniform(-random_bounds, random_bounds,size=(batch, N, N))
        sym_m = (m + np.transpose(m, axes=(0,2,1)))/2
        
        lst = np.concatenate((lst, sym_m), axis=0)
        lst = lst[np.all(np.abs(np.diagonal(lst, axis1 = 1, axis2 = 2)) < c, axis=1)]

    return lst

In [4]:
lst = gen(N, N_samples)

In [5]:
lst.shape

(50000, 5, 5)

### Eigenvalue method -- we test if -1 < eigvalues < 1

In [6]:
def eigen_way():
    
    lst_c = np.copy(lst)
    
    lst_c = lst_c[np.all(np.linalg.eigvals(lst_c) > -c, axis=1)]
    
    lst_c = lst_c[np.all(np.linalg.eigvals(lst_c) < c, axis=1)]
        
    return lst_c

#### Time execution

In [7]:
%timeit eigen_way()

123 ms ± 566 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


#### We list the survivors of the random matrices

In [8]:
lst_ei = eigen_way()

### Minor method -- we test if -1 < eigvalues < 1. Why ask for the whole diagonal? It trims even further matrices  as it is a necessary condition

In [9]:
def min_way():
    
    lst_c = np.copy(lst)
        
    lst_c = lst_c[np.all(np.abs(np.diagonal(lst_c, axis1 = 1, axis2 = 2)) < c, axis=1)]
    
    for k in range(2, N + 1):
        
        lst_c = lst_c[np.linalg.det((lst_c + c * np.eye(N))[:,:k,:k]) > 0]
        lst_c = lst_c[np.linalg.det((c * np.eye(N) - lst_c)[:,:k,:k]) > 0]
    
      
    return lst_c

#### Time execution is much faster, this without including simple conditions for the random generation of the matrices

In [10]:
%timeit min_way()

30.1 ms ± 1.25 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


#### We list the survivors of the random matrices

In [11]:
lst_min = min_way()

### The lists of both methods are the same

In [12]:
(lst_min == lst_ei).all()

True

### The surviving matrices

In [13]:
lst_min.shape

(537, 5, 5)